In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import dama as dm
#import tensorflow as tf
#import tensorflow_addons as tfa

from matplotlib.colors import LogNorm
from scipy import stats
from scipy.optimize import minimize
from scipy.optimize import curve_fit
from multiprocessing import Process, Pool
from spherical_opt import spherical_opt

In [ ]:
params = {'legend.fontsize': 17,
          'figure.figsize': (15, 9.3),
          'axes.labelsize': 24,
          'axes.titlesize': 24,
          'xtick.labelsize': 22,
          'ytick.labelsize': 22}
plt.rcParams.update(params)

par_names = ['x', 'y', 'z', 't', 'azi', 'zen', 'Ecscd', 'Etrck']
data_path = '../../../freedom/resources/toy_data/'
plot_path = '../../../plots/toy_model/'

def correct_azi(azi):
    azi = np.where(azi<-np.pi, azi+2*np.pi, azi)
    return np.where(azi>np.pi, azi-2*np.pi, azi)

def plot_diff(diff, bins, label):
    diff = diff[~np.isnan(diff)]
    plt.hist(diff, bins, label=label + ' (std=%.2f, iqr=%.2f)'%(np.std(diff), stats.iqr(diff)), alpha=0.5)
    if 'true' in label:
        o, c = 0.05, 'tab:Blue'
    else:
        o, c = 0.11, 'tab:Orange'

In [ ]:
from freedom.toy_model import advanced_toy_3D
detectors = np.vstack([[3.1,7.2,4.2,10.7,11.3,5.7,-1.0,-1.1], [-7.3,-6.7,3.5,2.7,-6.0,-10.5,-8.0,0.7], np.ones(8)]).T
detectors = np.repeat(detectors, 25, axis=0)
detectors[:, 2] = np.tile(np.linspace(-17, 17, 25), 8)
toy_experiment = advanced_toy_3D.advanced_toy_experiment(detectors=detectors, time_dist=advanced_toy_3D.pandel)

## Create/Get events

In [ ]:
N = 5000

#events, Truth = toy_experiment.generate_events(N)
#np.save('../../../freedom/resources/toy_data/toy_events_3D_test', events)
#np.save('../../../freedom/resources/toy_data/toy_truth_3D_test', Truth)

events = np.load('../../../freedom/resources/toy_data/toy_events_3D_test.npy', allow_pickle=True)[:N]
Truth = np.load('../../../freedom/resources/toy_data/toy_truth_3D_test.npy')[:N]

## Reco events

In [ ]:
bounds = np.array([[-2,12], [-12,4], [-18,18], [-30,30], [0, 2*np.pi], [0, np.pi], [1,20], [0,20]])

def init_points(hits, n_live_points, n_params=8, bound=bounds, seed=[None]):
    if seed[0] == None:
        avg = np.roll(np.average(hits[:, :4], axis=0), -1)
        low_lims = np.concatenate([avg-np.array([5,5,10,60]), np.array([0,0,1,0])])
        hig_lims = np.concatenate([avg+np.array([5,5,10,0]), np.array([2*np.pi,np.pi,30,30])])
    else:
        low_lims = seed - np.array([2,2,3,5,0.5,0.3,3,3])
        hig_lims = seed + np.array([2,2,3,5,0.5,0.3,3,3])
    
    uniforms = np.random.uniform(size=(n_live_points, n_params))
    initial_points = low_lims + uniforms * (hig_lims - low_lims)
    initial_points = np.clip(initial_points, bounds[:, 0], bounds[:, 1])
    return initial_points

#### true LLH

In [ ]:
def LLHs(X, event, only_c=False, only_h=False, fix=[None], bounds=bounds):
    if X.ndim == 1:
        return LLH(X, event, only_c, only_h, fix, bounds)
    else:
        o = []
        for x in X:
            o.append(LLH(x, event, only_c, only_h, fix, bounds))
        return np.array(o)

def LLH(X, event, only_c=False, only_h=False, fix=[None], bounds=bounds):
    #X: hypo_x, hypo_y, hypo_t, hypo_N_src, hypo_ang
    assert only_c + only_h < 2
    
    if fix[0] != None:
        X = np.insert(X, fix[0], fix[1])
        
    if ~np.alltrue(np.logical_and(bounds[:,0] <= X, X <= bounds[:,1]), axis=-1):
        return 1e9
    
    pos = np.array([X[0], X[1], X[2]])
    angs = np.array([X[4], X[5]])
    c_term = -toy_experiment.charge_term(event[0], pos, angs, X[6], X[7])
    h_term = -toy_experiment.hit_term(event[1], pos, X[3], angs, X[6], X[7])
    
    if only_c: return c_term
    if only_h: return h_term
    return c_term + h_term

In [ ]:
def fit_event(event):
    event, truth = event
    
    def eval_LLH(params):
        return LLHs(params, event)

    # seeding
    initial_points = init_points(event[1], 97) #, seed=truth
    
    # free fit
    fit_res = spherical_opt.spherical_opt(
        func=eval_LLH,
        method="CRS2", #Nelder-Mead
        initial_points=initial_points,
        rand=np.random.default_rng(42),
        spherical_indices=[[4,5]],
        batch_size=12,
        #**sph_opt_kwargs,
    )
    out = fit_res['x']
    #out = np.append(out, fit_res['fun'])
    '''
    # 1 param fixed fit
    initial_points = init_points(event[1], 97, seed=fit_res['x'])
    for i in range(8):
        def eval_LLH(params):
            params = np.delete(params, i)
            return LLH(params, event, fix=[i, truth[i]])
        m = spherical_opt.spherical_opt(func=eval_LLH, method="CRS2", initial_points=initial_points,
                                        rand=np.random.default_rng(42), spherical_indices=[[4,5]], batch_size=12)
        dllh = m['fun']-fit_res['fun']
        out = np.append(out, dllh)
    '''
    return list(out)

In [ ]:
%%time
with Pool(10) as p:
    outs = p.map(fit_event, zip(events, Truth))

In [ ]:
#np.save('../../../freedom/resources/toy_data/recos/toy_reco_3D_trueLLH_CRS_CoGSeed', outs)
Reco = np.array(outs) #np.load('../../../freedom/resources/toy_data/recos/toy_reco_3D_trueLLH_CRS_CoGSeed.npy')

#### NN LLH

In [ ]:
import math

from functools import partial
from freedom.llh_service.llh_service import LLHService
from freedom.llh_service.llh_client import LLHClient
from freedom.reco import crs_reco

In [ ]:
loc = '../../../freedom/resources/models/toy/'
service_conf = {
        "poll_timeout": 1,
        "flush_period": 1,
        "n_hypo_params": 8,
        "n_hit_features": 6,
        "n_evt_features": 2,
        "batch_size" : {
          "n_hypos": 200,
          "n_observations": 6000, 
        },
        "send_hwm": 10000,
        "recv_hwm": 10000,
        "hitnet_file": loc+'hitNet_3D.hdf5',
        "chargenet_file": loc+'chargeNet_3D.hdf5',
        "toy": True
}

In [ ]:
n_gpus = 4

base_req = "ipc:///tmp/recotestreq"
base_ctrl = "ipc:///tmp/recotestctrl"

req_addrs = []
ctrl_addrs = []
for i in range(n_gpus):
    req_addrs.append(f'{base_req}{i}')
    ctrl_addrs.append(f'{base_ctrl}{i}')
    
procs = []
for i in range(n_gpus):
    proc = Process(target=crs_reco.start_service, args=(service_conf, ctrl_addrs[i], req_addrs[i], i))
    proc.start()
    procs.append(proc)

In [ ]:
def fit_events(events, index, Truths, ctrl_addrs):
    outputs = []

    client = LLHClient(ctrl_addr=ctrl_addrs[index], conf_timeout=60000)
    def Eval_llh(params, event, fix=[None]):
        if fix[0] != None:
            params = np.insert(params, fix[0], fix[1])
            
        if ~np.alltrue(np.logical_and(bounds[:,0] <= params, params <= bounds[:,1]), axis=-1):
            return 1e9
        
        return client.eval_llh(event[1], event[0], params)

    for j, event in enumerate(events):
        def eval_LLH(params):
            if params.ndim == 1:
                return Eval_llh(params, event)
            else:
                o = []
                for p in params:
                    o.append(Eval_llh(p, event))
                return np.array(o)

        # seeding
        initial_points = init_points(event[1], 97) #, seed=Truths[j]
        
        #free fit
        fit_res = spherical_opt.spherical_opt(
            func=eval_LLH,
            method="CRS2", #Nelder-Mead
            initial_points=initial_points,
            rand=np.random.default_rng(42),
            spherical_indices=[[4,5]],
            batch_size=12,
            #**sph_opt_kwargs,
        )
        out = fit_res['x']
        #out = np.append(out, fit_res['fun'])
        '''
        # 1 param fixed fit
        initial_points = init_points(event[1], 97, seed=fit_res['x'])
        for i in range(8):
            def eval_LLH(params):
                params = np.delete(params, i)
                return Eval_llh(params, event, fix=[i, Truths[j][i]])
            m = spherical_opt.spherical_opt(func=eval_LLH, method="CRS2", initial_points=initial_points,
                                            rand=np.random.default_rng(42), spherical_indices=[[4,5]], batch_size=12)
            dllh = m['fun']-fit_res['fun']
            out = np.append(out, dllh)
        '''
        outputs.append(out)
        
    return outputs

In [ ]:
events_to_process = len(events)
pool_size = 200
evts_per_proc = int(math.ceil(events_to_process/pool_size))
evt_splits = [events[i*evts_per_proc:(i+1)*evts_per_proc] for i in range(pool_size)]
true_splits = [Truth[i*evts_per_proc:(i+1)*evts_per_proc] for i in range(pool_size)]
print(sum(len(l) for l in evt_splits))

gpu_inds = np.arange(pool_size) % n_gpus

fit_events_partial = partial(
        fit_events,
        ctrl_addrs=ctrl_addrs
)

In [ ]:
%%time
# reconstruct with a worker pool; one LLH client per worker
with Pool(pool_size) as p:
    outs = p.starmap(fit_events_partial, zip(evt_splits, gpu_inds, true_splits))

all_outs = sum((out for out in outs), [])
all_outs = np.array(all_outs).reshape((events_to_process, 8)) #9 16

In [ ]:
#np.save('../../../freedom/resources/toy_data/recos/toy_reco_3D_nnLLH_CRS_CoGSeed', all_outs)
Reco_NN = np.array(all_outs) #np.load('../../../freedom/resources/toy_data/recos/toy_reco_3D_nnLLH_CRS_CoGSeed.npy')

In [ ]:
# kill all the services
import zmq
for proc, ctrl_addr in zip(procs, ctrl_addrs): 
    with zmq.Context.instance().socket(zmq.REQ) as ctrl_sock:
        ctrl_sock.connect(ctrl_addr)
        ctrl_sock.send_string("die")
        proc.join()

#### plots

In [ ]:
#Reco, dLLH = np.split(np.load(data_path+'toy_reco_3D_trueLLH_CRS_randomSeed.npy'), 2, axis=1)
#Reco_NN, dLLH_NN = np.split(np.load(data_path+'toy_reco_3D_nnLLH_CRS_randomSeed.npy'), 2, axis=1)
#Reco = np.load(data_path+'toy_reco_3D_trueLLH_CRS_randomSeed.npy')
#Reco_NN = np.load(data_path+'toy_reco_3D_nnLLH_CRS_randomSeed.npy')

#Truth = np.load('../../../freedom/resources/toy_data/toy_truth_3D_test.npy')[:len(Reco)]

label1, label2 = 'true LLH', 'NN LLH'

In [ ]:
fig = plt.figure(figsize=(20, 17))

for i in range(8):
    plt.subplot(3,3,i+1)
    plt.hist(Reco[:, i], 100, label=label1, histtype='step', linewidth=2)
    plt.hist(Reco_NN[:, i], 100, label=label2, histtype='step', linewidth=2)
    plt.hist(Truth[:, i], 100, label='Truth', histtype='step', linewidth=2)
    plt.legend()
    plt.xlabel(par_names[i])
    if i in [3, 6, 7]:
        plt.yscale('log')
#plt.savefig(plot_path+'/reco_dists/param_dists_3D_CRS_randomSeed', bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(20, 17))
plt.suptitle('Reco - Truth', size=20, y=.91)

plt.subplot(3,3,1)
plt.axvline(0, color='black', linestyle='--')
plot_diff(Reco[:, 0]-Truth[:, 0], np.linspace(-3,3,50), label1)
plot_diff(Reco_NN[:, 0]-Truth[:, 0], np.linspace(-3,3,50), label2)
plt.legend()
plt.xlabel('x')

plt.subplot(3,3,2)
plt.axvline(0, color='black', linestyle='--')
plot_diff(Reco[:, 1]-Truth[:, 1], np.linspace(-3,3,50), label1)
plot_diff(Reco_NN[:, 1]-Truth[:, 1], np.linspace(-3,3,50), label2)
plt.legend()
plt.xlabel('y')

plt.subplot(3,3,3)
plt.axvline(0, color='black', linestyle='--')
plot_diff(Reco[:, 2]-Truth[:, 2], np.linspace(-2,2,50), label1)
plot_diff(Reco_NN[:, 2]-Truth[:, 2], np.linspace(-2,2,50), label2)
plt.legend()
plt.xlabel('z')

plt.subplot(3,3,4)
plt.axvline(0, color='black', linestyle='--')
plot_diff(Reco[:, 3]-Truth[:, 3], np.linspace(-5,5,50), label1)
plot_diff(Reco_NN[:, 3]-Truth[:, 3], np.linspace(-5,5,50), label2)
plt.legend()
plt.xlabel('t')

plt.subplot(3,3,5)
plt.axvline(0, color='black', linestyle='--')
plot_diff(correct_azi(Reco[:, 4]-Truth[:, 4]), np.linspace(-2,2,50), label1)
plot_diff(correct_azi(Reco_NN[:, 4]-Truth[:, 4]), np.linspace(-2,2,50), label2)
plt.legend()
plt.xlabel('azi')

plt.subplot(3,3,6)
plt.axvline(0, color='black', linestyle='--')
plot_diff(Reco[:, 5]-Truth[:, 5], np.linspace(-1,1,50), label1)
plot_diff(Reco_NN[:, 5]-Truth[:, 5], np.linspace(-1,1,50), label2)
plt.legend()
plt.xlabel('zen')

plt.subplot(3,3,7)
plt.axvline(0, color='black', linestyle='--')
plot_diff(Reco[:, 6]-Truth[:, 6], np.linspace(-20,20,50), label1)
plot_diff(Reco_NN[:, 6]-Truth[:, 6], np.linspace(-20,20,50), label2)
plt.legend()
plt.xlabel('Ecscd')

plt.subplot(3,3,8)
plt.axvline(0, color='black', linestyle='--')
plot_diff(Reco[:, 7]-Truth[:, 7], np.linspace(-20,20,50), label1)
plot_diff(Reco_NN[:, 7]-Truth[:, 7], np.linspace(-20,20,50), label2)
plt.legend()
plt.xlabel('Etrck')

#plt.savefig(plot_path+'/reco_dists/reco_dists_3D_NM_CoGSeed', bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(23, 17))

cscds = Truth[:, 7] == 0
for i in range(8):
    plt.subplot(3,3,i+1)
    plt.plot([min(Truth[:, i]), max(Truth[:, i])], [min(Truth[:, i]), max(Truth[:, i])], color='black', linestyle='--')
    #plt.hist2d(Truth[:, i], Reco[:, i], np.linspace(bounds[i][0], bounds[i][1], 20), norm=LogNorm())
    plt.hist2d(Truth[:, i], Reco_NN[:, i], np.linspace(bounds[i][0], bounds[i][1], 20), norm=LogNorm())
    plt.colorbar()
    #plt.legend()
    plt.title(par_names[i])
    if i == 7: plt.xlabel('Truth')
    if i == 3: plt.ylabel('Reco')
#plt.text('Reco')

#plt.savefig(plot_path+'/reco_dists/reco_truth_3D_trueLLH_CRS_randomSeed', bbox_inches='tight')

In [ ]:
#plt.hist2d(Reco[:, 0], Reco[:, 1], [np.linspace(bounds[0][0], bounds[0][1], 30), 
#                                    np.linspace(bounds[1][0], bounds[1][1], 30)])
#plt.colorbar()

plt.hist(np.cos(Truth[~cscds, 5]), 50, histtype='step')
plt.hist(np.cos(Reco[~cscds, 5]), 50, histtype='step');

## dLLH

In [ ]:
i = 6

if i == 4:
    resi = np.abs(correct_azi(Reco_NN[:,i]-Truth[:,i])) #
else:
    resi = np.abs(Reco_NN[:,i]-Truth[:,i]) #
var = par_names[i]

dllh = np.nan_to_num(dLLH_NN[:,i]) #
dllh_50 = np.percentile(dllh[dllh > 0], 50)
dllh_68 = np.percentile(dllh[dllh > 0], 68)

print(len(dllh[dllh > 0])/len(dllh))

In [ ]:
plt.title(var)
plt.hist(dllh, np.linspace(-20,20,200)) #
plt.axvline(dllh_68, color='red', linestyle='--', label='68 percentile %.2f'%(dllh_68))
plt.axvline(0, color='black', linestyle=':')
plt.legend()
plt.xlabel(r'$LLH_{7p} - LLH_{8p}$')
plt.ylabel('#events')
plt.yscale('log')
#plt.savefig('../../../plots/toy_model/dLLH/dLLH_'+var+'_nnLLH', bbox='tight') #+'_truthSeeded'

In [ ]:
dllh_range = np.logspace(-1.5, 1, 10) #-2, 0.8
dllh_center = (dllh_range[1:]+dllh_range[:-1])/2
medi, mean = [], []
for i in range(len(dllh_range)-1):
    test = resi[(dllh > dllh_range[i]) & (dllh < dllh_range[i+1])]
    if len(test) == 0: test = np.zeros(1)
    mean.append(np.mean(test))
    medi.append(np.median(test))

In [ ]:
def func(x, a): #, b
    return a * np.sqrt(x) #+ b

popt_medi, pcov = curve_fit(func, dllh_center, medi)
unc_50_medi = func(dllh_50, popt_medi[0]) #, popt_medi[1]

popt_mean, pcov = curve_fit(func, dllh_center, mean)
unc_68_mean = func(dllh_68, popt_mean[0]) #, popt_mean[1]

popt_medi[0], popt_mean[0]

In [ ]:
plt.scatter(dllh, resi)

plt.scatter(dllh_center, mean, color='r', label='means')
plt.plot(dllh_center, func(dllh_center, popt_mean[0]), color='r', label=r'$a\cdot\sqrt{x}$ fit to means')
plt.axvline(dllh_68, color='red', linestyle='--', label='68 percentile (%.2f, %.2f)'%(dllh_68, unc_68_mean))
plt.axhline(unc_68_mean, color='red', linestyle='--')
'''
plt.scatter(dllh_center, medi, color='black', label='medians')
plt.plot(dllh_center, func(dllh_center, popt_medi[0]), color='black', label=r'$a\cdot\sqrt{x}$ fit to medians')
plt.axvline(dllh_50, color='black', linestyle='--', label='50 percentile (%.2f, %.2f)'%(dllh_50, unc_50_medi))
plt.axhline(unc_50_medi, color='black', linestyle='--')
'''
#plt.axvline(dllh_range[0], color='black')
#plt.axvline(dllh_range[-1], color='black')

plt.title(var)
plt.legend()
plt.loglog()
plt.xlim(1e-3, 1e2)
plt.xlabel(r'$LLH_{7p} - LLH_{8p}$')
plt.ylim(1e-3, 150)
plt.ylabel('|reco - true|')
#plt.savefig('../../../plots/toy_model/dLLH/unc_'+var+'_nnLLH', bbox='tight') #+'_truthSeeded'